# Connors Reversal Strategy Batch Optimization

这个notebook用于对多个交易对进行Connors Reversal策略的批量优化。

In [1]:
# 导入必要的库
import os
import pandas as pd
from datetime import datetime
from glob import glob
import numpy as np
from backtesting import Backtest
from ConnorsReversalStrategy import ConnorsReversal
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\future'):
    """
    加载并重采样期货数据
    """
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 准备存储所有数据的列表
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 遍历每一天
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并所有数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    
    # 设置时间索引
    combined_df.set_index('datetime', inplace=True)
    
    # 重采样到目标时间周期
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    # 转换为backtesting.py格式
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    
    # 最终清理
    backtesting_df = backtesting_df.dropna()
    
    return backtesting_df

In [3]:
def get_all_symbols(data_path, date_str):
    """获取指定日期目录下的所有交易对"""
    daily_path = os.path.join(data_path, date_str)
    if not os.path.exists(daily_path):
        return []
    
    files = glob(os.path.join(daily_path, f"{date_str}_*_USDT_1m.csv"))
    symbols = []
    for file in files:
        # 从文件名提取交易对名称
        filename = os.path.basename(file)
        symbol = filename.split('_')[1]
        if symbol not in symbols:
            symbols.append(symbol)
    return symbols

def verify_data_completeness(symbol, start_date, end_date, data_path):
    """验证交易对在指定时间段内的数据完整性"""
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        file_path = os.path.join(data_path, date_str, f"{date_str}_{symbol}_USDT_1m.csv")
        if not os.path.exists(file_path):
            return False
    return True

In [4]:
def optimize_strategy(symbol, start_date, end_date, data_path):
    """对单个交易对进行策略优化"""
    try:
        print(f"开始加载 {symbol} 的数据...")
        # 加载数据
        df = load_and_resample_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            source_timeframe='1m',
            target_timeframe='30min'
        )
        # print(f"数据加载完成, 共 {len(df)} 条记录")
        # print(f"数据时间范围: {df.index[0]} 到 {df.index[-1]}")
        # print(f"数据列: {df.columns.tolist()}")
        # 
        # print("\n初始化回测实例...")
        # 初始化回测实例
        bt = Backtest(
            df,
            ConnorsReversal,
            commission=.0004,
            margin=1/8,
            trade_on_close=True,
            exclusive_orders=True,
            hedging=False
        )
        # print("回测实例初始化完成")
        
        print("\n开始参数优化...")
        # print("优化参数范围:")
        # print("- lowest_point_bars: 5-50, 步长5")
        # print("- rsi_length: 2-50, 步长5")
        # print("- sell_barrier: 65-85, 步长5")
        # print("- dca_parts: 4-11, 步长1")

        # 定义优化参数范围并使用sambo方法
        opt_results = bt.optimize(
            lowest_point_bars=range(5, 51, 5),
            rsi_length=range(2, 51, 5),
            sell_barrier=range(65, 86, 5),
            dca_parts=range(4, 12, 1),
            maximize='Return [%]',
            constraint=lambda p: True,
            method='sambo',
            max_tries=200,
            random_state=0
        )
        
        # 扩展结果字典
        result = {
            'Symbol': symbol,
            'Initial Capital': 10000,
            'Lowest Point Bars': opt_results._strategy.lowest_point_bars,
            'RSI Length': opt_results._strategy.rsi_length,
            'Sell Barrier': opt_results._strategy.sell_barrier,
            'DCA Parts': opt_results._strategy.dca_parts,
            'Total Return (%)': opt_results['Return [%]'],
            'Sharpe Ratio': opt_results['Sharpe Ratio'],
            'Max Drawdown (%)': opt_results['Max. Drawdown [%]'],
            'Win Rate (%)': opt_results['Win Rate [%]'],
            'Total Trades': opt_results['# Trades'],
            'Optimization Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        # 立即保存当前结果到Excel
        results_file = 'optimization_results.xlsx'
        
        try:
            # 尝试读取现有文件
            existing_df = pd.read_excel(results_file)
            # 添加新结果
            new_df = pd.concat([existing_df, pd.DataFrame([result])], ignore_index=True)
        except FileNotFoundError:
            # 如果文件不存在，创建新的DataFrame
            new_df = pd.DataFrame([result])
        
        # 保存更新后的结果
        new_df.to_excel(results_file, index=False)
        # print(f"\n已将{symbol}的优化结果保存到: {os.path.abspath(results_file)}")
        
        # 打印优化结果
        # print("\n=== 最优参数 ===")
        # print(f"最低点周期: {result['Lowest Point Bars']}")
        # print(f"RSI周期: {result['RSI Length']}")
        # print(f"RSI卖出阈值: {result['Sell Barrier']}")
        # print(f"DCA分批次数: {result['DCA Parts']}")
        
        # print("\n=== 回测结果 ===")
        # print(f"总收益率: {result['Total Return (%)']}%")
        # print(f"夏普比率: {result['Sharpe Ratio']}")
        # print(f"最大回撤: {result['Max Drawdown (%)']}%")
        # print(f"胜率: {result['Win Rate (%)']}%")
        # print(f"总交易次数: {result['Total Trades']}")
        
        return result
        
    except Exception as e:
        print(f"优化{symbol}时出错: {str(e)}")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误详情: {str(e)}")
        import traceback
        print("完整错误追踪:")
        print(traceback.format_exc())
        return None

# 主执行代码部分保持不变，因为optimize_strategy函数已经处理了结果的保存

In [5]:
# 设置参数
data_path = r'\\znas\Main\future'
start_date = '2024-01-01'
end_date = '2025-01-01'

print(f"使用数据路径: {data_path}")
print(f"回测时间范围: {start_date} 到 {end_date}")

# 获取所有交易对
symbols = get_all_symbols(data_path, '2024-01-01')
print(f"找到 {len(symbols)} 个交易对:")
print("\n".join(symbols))

# 存储结果的列表
results = []

# 对每个交易对进行处理
for i, symbol in enumerate(symbols, 1):
    print(f"\n正在处理第 {i}/{len(symbols)} 个交易对: {symbol}")
    print("-" * 50)
    
    # 验证数据完整性
    print(f"验证 {symbol} 的数据完整性...")
    if not verify_data_completeness(symbol, start_date, end_date, data_path):
        print(f"警告: 跳过 {symbol} - 数据不完整")
        continue
    print(f"{symbol} 数据完整性验证通过")
    
    # 进行优化
    print(f"开始优化 {symbol}...")
    result = optimize_strategy(symbol, start_date, end_date, data_path)
    if result:
        results.append(result)
        print(f"优化结果:")
        for key, value in result.items():
            print(f"  {key}: {value}")
        print(f"完成 {symbol} 的优化")
    else:
        print(f"警告: {symbol} 优化失败")

# 将结果保存到Excel
if results:
    df_results = pd.DataFrame(results)
    output_file = 'optimization_results.xlsx'
    # 保存 Excel 文件
    df_results.to_excel(output_file, index=False)
    print(f"\n成功完成 {len(results)}/{len(symbols)} 个交易对的优化")
    print(f"结果已保存到 {output_file}")
    print("\n优化结果统计:")
    print(df_results.describe())
else:
    print("\n警告: 没有找到任何有效的优化结果")

使用数据路径: \\znas\Main\future
回测时间范围: 2024-01-01 到 2025-01-01
找到 45 个交易对:
ADAUSDT
ALGOUSDT
ATOMUSDT
BALUSDT
BANDUSDT
BATUSDT
BCHUSDT
BNBUSDT
BTCUSDT
COMPUSDT
CRVUSDT
DASHUSDT
DEFIUSDT
DOGEUSDT
DOTUSDT
EOSUSDT
ETCUSDT
ETHUSDT
IOSTUSDT
IOTAUSDT
KASUSDT
KAVAUSDT
KNCUSDT
LINKUSDT
LTCUSDT
MKRUSDT
NEOUSDT
OMGUSDT
ONTUSDT
QTUMUSDT
RLCUSDT
SNXUSDT
SXPUSDT
THETAUSDT
TRXUSDT
VETUSDT
WAVESUSDT
XLMUSDT
XMRUSDT
XRPUSDT
XTZUSDT
YFIUSDT
ZECUSDT
ZILUSDT
ZRXUSDT

正在处理第 1/45 个交易对: ADAUSDT
--------------------------------------------------
验证 ADAUSDT 的数据完整性...
ADAUSDT 数据完整性验证通过
开始优化 ADAUSDT...
开始加载 ADAUSDT 的数据...

开始参数优化...
['c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\python310.zip', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\DLLs', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting', '', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32', 'c:\\Users\\

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

优化结果:
  Symbol: ADAUSDT
  Initial Capital: 10000
  Lowest Point Bars: 19
  RSI Length: 5
  Sell Barrier: 65
  DCA Parts: 6
  Total Return (%): 1083.9879936356133
  Sharpe Ratio: 0.5419700428690508
  Max Drawdown (%): -72.53127825720777
  Win Rate (%): 57.76772247360482
  Total Trades: 663
  Optimization Date: 2025-02-02 00:17:18
完成 ADAUSDT 的优化

正在处理第 2/45 个交易对: ALGOUSDT
--------------------------------------------------
验证 ALGOUSDT 的数据完整性...
ALGOUSDT 数据完整性验证通过
开始优化 ALGOUSDT...
开始加载 ALGOUSDT 的数据...

开始参数优化...
['c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\python310.zip', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\DLLs', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting', '', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32\\lib', 'c:\\Users\\x7498\\anacond

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

优化结果:
  Symbol: ALGOUSDT
  Initial Capital: 10000
  Lowest Point Bars: 48
  RSI Length: 3
  Sell Barrier: 85
  DCA Parts: 10
  Total Return (%): 272.10144074960385
  Sharpe Ratio: 0.7437847974949119
  Max Drawdown (%): -51.9154020353656
  Win Rate (%): 52.77777777777778
  Total Trades: 360
  Optimization Date: 2025-02-02 00:18:38
完成 ALGOUSDT 的优化

正在处理第 3/45 个交易对: ATOMUSDT
--------------------------------------------------
验证 ATOMUSDT 的数据完整性...
ATOMUSDT 数据完整性验证通过
开始优化 ATOMUSDT...
开始加载 ATOMUSDT 的数据...

开始参数优化...
['c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\python310.zip', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\DLLs', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting', '', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32\\lib', 'c:\\Users\\x7498\\anaco

...

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

优化结果:
  Symbol: ZILUSDT
  Initial Capital: 10000
  Lowest Point Bars: 39
  RSI Length: 3
  Sell Barrier: 80
  DCA Parts: 9
  Total Return (%): 603.8053308793261
  Sharpe Ratio: 0.9791962156248596
  Max Drawdown (%): -53.884146735083704
  Win Rate (%): 56.4625850340136
  Total Trades: 441
  Optimization Date: 2025-02-02 04:24:05
完成 ZILUSDT 的优化

正在处理第 45/45 个交易对: ZRXUSDT
--------------------------------------------------
验证 ZRXUSDT 的数据完整性...
ZRXUSDT 数据完整性验证通过
开始优化 ZRXUSDT...
开始加载 ZRXUSDT 的数据...

开始参数优化...
['c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\python310.zip', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\DLLs', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting', '', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32', 'c:\\Users\\x7498\\anaconda3\\envs\\backtesting\\lib\\site-packages\\win32\\lib', 'c:\\Users\\x7498\\anaconda3\\e

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

优化结果:
  Symbol: ZRXUSDT
  Initial Capital: 10000
  Lowest Point Bars: 25
  RSI Length: 2
  Sell Barrier: 85
  DCA Parts: 11
  Total Return (%): 54.148623531603135
  Sharpe Ratio: 0.360781812966692
  Max Drawdown (%): -72.75081510721846
  Win Rate (%): 58.20668693009119
  Total Trades: 658
  Optimization Date: 2025-02-02 04:30:02
完成 ZRXUSDT 的优化

成功完成 45/45 个交易对的优化
结果已保存到 optimization_results.xlsx

优化结果统计:
       Initial Capital  Lowest Point Bars  RSI Length  Sell Barrier  \
count             45.0          45.000000   45.000000     45.000000   
mean           10000.0          35.644444   10.066667     75.844444   
std                0.0          13.258197   13.146724      6.986856   
min            10000.0           5.000000    2.000000     65.000000   
25%            10000.0          27.000000    3.000000     69.000000   
50%            10000.0          39.000000    4.000000     77.000000   
75%            10000.0          47.000000   10.000000     81.000000   
max            10000.0  

: 